In [1]:
import math
import torchtext
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab,Vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torchtext.transforms import ToTensor
from torch import Tensor
from torch.nn import TransformerEncoder,TransformerDecoder,TransformerEncoderLayer,TransformerDecoderLayer
import io
import time

In [2]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.fr.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

In [3]:
train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

In [4]:
import spacy

In [5]:
import pandas as pd
df = pd.read_csv('questions_easy.csv')
fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [7]:
counter = Counter()
for string_ in df['en']:
    counter.update(en_tokenizer(string_))

en_vocab =  vocab(counter,specials=['<unk>','<pad>','<bos>','<eos>'])

counter = Counter()
for string_ in df['fr']:
    counter.update(fr_tokenizer(string_))

fr_vocab =  vocab(counter,specials=['<unk>','<pad>','<bos>','<eos>'])

fr_vocab.set_default_index(0)
en_vocab.set_default_index(0)

In [8]:
def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0],encoding='utf8'))
    raw_en_iter = iter(io.open(filepaths[1],encoding='utf8'))
    fr_data = []
    en_data = []
    for (raw_de,raw_en) in zip(raw_de_iter,raw_en_iter):
            de_data.append([2]+de_vocab(de_tokenizer(raw_de.rstrip('n')))+[3])
            en_data.append([2]+en_vocab(en_tokenizer(raw_en.rstrip('n')))+[3])
    return  fr_data,en_data    

In [9]:
raw_fr_iter = iter(df['fr'])
raw_en_iter = iter(df['en'])
fr_data = []
en_data = []
for (raw_fr,raw_en) in zip(raw_fr_iter,raw_en_iter):
        fr_data.append([2]+fr_vocab(fr_tokenizer(raw_fr.rstrip('n')))+[3])
        en_data.append([2]+en_vocab(en_tokenizer(raw_en.rstrip('n')))+[3])
train_data = fr_data, en_data
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [10]:
to_tensor = ToTensor(padding_value=1)

In [11]:
fr_train_data = to_tensor(train_data[0])
en_train_data = to_tensor(train_data[1])
fr_val_data = to_tensor(val_data[0])
en_val_data = to_tensor(val_data[1])
fr_test_data = to_tensor(test_data[0])
en_test_data = to_tensor(test_data[1])

In [12]:
fr_train_data.size()

torch.Size([29000, 47])

In [13]:
en_train_data.size()

torch.Size([29000, 44])

In [14]:
train_ds = TensorDataset(fr_train_data,en_train_data)
val_ds = TensorDataset(fr_val_data,en_val_data)
test_ds = TensorDataset(fr_test_data,en_test_data)

In [15]:
bs = 128

In [16]:
train_dl = DataLoader(train_ds,batch_size=bs,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=bs,shuffle=True)
test_dl = DataLoader(test_ds,batch_size=bs,shuffle=True)

In [17]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super().__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward,batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward,batch_first=True)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory,tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)
    
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
    
class PositionalEncoding(nn.Module):
    
    def __init__(self,emb_size,dropout,max_len=5000):
        super().__init__()
        den = torch.exp(-torch.arange(0,emb_size,2) * math.log(10000)/emb_size)
        pos = torch.arange(0,max_len).reshape(max_len,1)
        pos_embedding = torch.zeros((max_len,emb_size))
        pos_embedding[:,0::2] = torch.sin(pos * den)
        pos_embedding[:,1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding',pos_embedding)
        
    def forward(self,token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0),:])
                                      
                                      
        
        

In [18]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [19]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz,sz),device='cuda')) == 1).transpose(0,1)
    
    mask = mask.float().masked_fill(mask==0,float('-inf')).masked_fill(mask==1,float(0.0))
    return mask

In [20]:
def create_mask(src,tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]
    
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len).cuda()
    src_mask = torch.zeros((src_seq_len,src_seq_len),device='cuda').type(torch.bool)
    
    src_padding_mask = (src == 1).cuda()
    tgt_padding_mask = (tgt == 1).cuda()
    
    return src_mask, tgt_mask, src_padding_mask,tgt_padding_mask

In [21]:
SRC_VOCAB_SIZE = len(fr_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 64
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 50
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [37]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)


In [38]:
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [39]:
transformer = transformer.cuda()

In [40]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)

optimizer = torch.optim.Adam(transformer.parameters(),lr=0.0001,betas=(0.9,0.98),eps=1e-9)

In [41]:
def train_epoch(model, train_dl, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_dl):
        src = src.cuda()
        tgt = tgt.cuda()
        
        tgt_input = tgt[:,:-1]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask,
                tgt_mask, src_padding_mask,
                tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[:, 1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(train_dl)

        

In [42]:
def evaluate(model,val_dl):
    model.eval()
    losses = 0
    
    for idx, (src,tgt) in enumerate(val_dl):
        src = src.cuda()
        tgt = tgt.cuda()
        
        tgt_input =tgt[:,:-1]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask,
                tgt_mask, src_padding_mask,
                tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[:,1:]
        
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        
        losses += loss.item()
        
    return losses/len(val_dl)

In [43]:
for epoch in range(19):
    start_time = time.time()
    train_loss = train_epoch(transformer,train_dl,optimizer)
    
    end_time = time.time()
    
    val_loss = evaluate(transformer,val_dl)
    
    print(f'Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, '
         f'Epoch time = {(end_time - start_time):.3f}s ')

Epoch: 0, Train loss: 5.233, Val loss: 3.972, Epoch time = 70.432s 
Epoch: 1, Train loss: 3.719, Val loss: 3.433, Epoch time = 73.081s 
Epoch: 2, Train loss: 3.308, Val loss: 3.150, Epoch time = 74.110s 
Epoch: 3, Train loss: 3.018, Val loss: 2.931, Epoch time = 73.863s 
Epoch: 4, Train loss: 2.764, Val loss: 2.727, Epoch time = 75.674s 
Epoch: 5, Train loss: 2.534, Val loss: 2.559, Epoch time = 75.814s 
Epoch: 6, Train loss: 2.341, Val loss: 2.428, Epoch time = 77.545s 
Epoch: 7, Train loss: 2.176, Val loss: 2.332, Epoch time = 78.984s 
Epoch: 8, Train loss: 2.034, Val loss: 2.266, Epoch time = 79.218s 
Epoch: 9, Train loss: 1.906, Val loss: 2.193, Epoch time = 81.111s 
Epoch: 10, Train loss: 1.793, Val loss: 2.170, Epoch time = 81.286s 
Epoch: 11, Train loss: 1.691, Val loss: 2.134, Epoch time = 80.992s 
Epoch: 12, Train loss: 1.596, Val loss: 2.094, Epoch time = 80.932s 
Epoch: 13, Train loss: 1.509, Val loss: 2.099, Epoch time = 81.206s 
Epoch: 14, Train loss: 1.428, Val loss: 2.06

In [29]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.cuda()
    src_mask = src_mask.cuda()
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(2).type(torch.long).cuda()
    for i in range(max_len-1):

        memory = memory.cuda()
        memory_mask = torch.zeros(ys.shape[1], memory.shape[1]).cuda().type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                                    .type(torch.bool)).cuda()
        out = model.decode(ys, memory, tgt_mask).squeeze(0)[-1]
        prob = model.generator(out)
        next_word = prob.argmax()
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == 3:
              break
    return ys


def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    tokens = [2]+de_vocab(de_tokenizer(src))+[3]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(1,num_tokens))
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=2).flatten()
    return " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [45]:
output = translate(transformer, "Eine Gruppe von Menschen steht vor einem Flughafen .", de_vocab, en_vocab, de_tokenizer)
print(output)

 A group of people standing in front of an airport . 
 
